# Rollout

- Ref: https://learnk8s.io/kubernetes-rollbacks
- Ref: https://ithelp.ithome.com.tw/articles/10195639

滾動升級/回滾可以讓你在不終止現有服務情況下，進行升級。



In [ ]:
# makesure minikube is running
! minikube start

😄  minikube v1.28.0 on Ubuntu 22.04 (vbox/amd64)
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🏃  Updating the running docker "minikube" container ...
🐳  Preparing Kubernetes v1.25.3 on Docker 20.10.20 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


In [ ]:
# get nodes
! kubectl get nodes

NAME       STATUS   ROLES           AGE     VERSION
minikube   Ready    control-plane   6d18h   v1.25.3


In [ ]:
# get current contexts
! kubectl config get-contexts

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   default


In [ ]:
# delete all stuff
! kubectl delete service --all
! kubectl delete pods --all
! kubectl delete replica --all
! kubectl delete deployment --all

service "kubernetes" deleted
pod "hostname-5477bcd76c-gbchf" deleted
pod "hostname-5477bcd76c-l8lmk" deleted
pod "hostname-5477bcd76c-th95n" deleted
pod "hostname-5477bcd76c-wq4rl" deleted
pod "hostname-5477bcd76c-z6bdp" deleted
pod "hostname-5477bcd76c-zhd6n" deleted
^C
error: the server doesn't have a resource type "replica"
No resources found


In [ ]:
! kubectl delete deployment --all
! kubectl create deployment hostname --replicas=3 --image=brokenpen/hostname:v1 

deployment.apps "hostname" deleted
deployment.apps/hostname created


In [ ]:
! kubectl get deployment hostname --show-labels  -o wide

NAME       READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                  SELECTOR       LABELS
hostname   0/3     3            0           1s    hostname     brokenpen/hostname:v1   app=hostname   app=hostname


In [ ]:
! kubectl get replicaset -o wide

NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-7c99c74d96   3         3         0       1s    hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


In [ ]:
! kubectl get pod -o wide

NAME                        READY   STATUS              RESTARTS   AGE    IP            NODE       NOMINATED NODE   READINESS GATES
hostname-5477bcd76c-lsswh   1/1     Terminating         0          117s   172.17.0.10   minikube   <none>           <none>
hostname-5477bcd76c-n762q   1/1     Terminating         0          115s   172.17.0.11   minikube   <none>           <none>
hostname-5477bcd76c-sxgl9   1/1     Terminating         0          2m     172.17.0.9    minikube   <none>           <none>
hostname-7c99c74d96-gp8jd   0/1     ContainerCreating   0          1s     <none>        minikube   <none>           <none>
hostname-7c99c74d96-h72z2   0/1     ContainerCreating   0          1s     <none>        minikube   <none>           <none>
hostname-7c99c74d96-rzfqx   0/1     ContainerCreating   0          1s     <none>        minikube   <none>           <none>
hostname-7d756bf846-t7dcs   0/1     Terminating         0          66s    172.17.0.3    minikube   <none>           <none>


<img class="img" style="width:480px;" src="https://learnk8s.io/a/e2d1c566c4b5cd59d9fa8e0de547344b.svg"></img>

In [ ]:
! kubectl set image deployment/hostname \
    hostname=brokenpen/hostname:v2

deployment.apps/hostname image updated


In [ ]:
! kubectl get rs -o wide 
! sleep 2; echo; echo;
! kubectl get rs -o wide
! sleep 2; echo; echo;
! kubectl get rs -o wide
! sleep 2; echo; echo;
! kubectl get rs -o wide
! sleep 2; echo; echo;

NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   1         1         0       1s    hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   3         3         3       5s    hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   2         2         1       3s    hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   2         2         2       7s    hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   3         3         2       5s    hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c7

<img class="img" style="width:480px;" src="img/rollout-ready-2.png"></img>
---
<img class="img" style="width:480px;" src="img/rollout-ready-3.png"></img>
---
<img class="img" style="width:480px;" src="img/rollout-ready-4.png"></img>
---
<img class="img" style="width:480px;" src="img/rollout-ready-5.png"></img>

In [ ]:
! kubectl rollout status deployment/hostname

Waiting for deployment "hostname" rollout to finish: 1 out of 3 new replicas have been updated...


In [ ]:
! kubectl get deployments hostname -o wide

In [ ]:
# image v2 的 replciaSet 已經建立
! kubectl get rs -o wide

NAME                  DESIRED   CURRENT   READY   AGE    CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   3         3         3       15s    hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   0         0         0       100s   hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


In [ ]:
! kubectl get pods 

NAME                        READY   STATUS        RESTARTS   AGE    IP           NODE       NOMINATED NODE   READINESS GATES
hostname-5477bcd76c-9pr8x   1/1     Running       0          19s    172.17.0.6   minikube   <none>           <none>
hostname-5477bcd76c-f6zs8   1/1     Running       0          17s    172.17.0.7   minikube   <none>           <none>
hostname-5477bcd76c-n77px   1/1     Running       0          14s    172.17.0.8   minikube   <none>           <none>
hostname-7c99c74d96-d6sts   1/1     Terminating   0          104s   172.17.0.3   minikube   <none>           <none>
hostname-7c99c74d96-k4pbr   1/1     Terminating   0          104s   172.17.0.4   minikube   <none>           <none>
hostname-7c99c74d96-pmtzt   1/1     Terminating   0          104s   172.17.0.5   minikube   <none>           <none>
